In [ ]:
import requests
import json
from datetime import date, timedelta

In [0]:

# Mount ADLS Gen2
# Required each time the cluster is restarted which should be only on the first notebook as they run in order

tiers = ["bronze", "silver", "gold"]
adls_paths = {tier: f"abfss://{tier}@dataearthquakee.dfs.core.windows.net/" for tier in tiers}  ##substitue with adl path. Can be seen on the overview on azure adl

# Accessing paths
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"] 

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

[FileInfo(path='abfss://gold@dataearthquakee.dfs.core.windows.net/earthquake_events_gold/', name='earthquake_events_gold/', size=0, modificationTime=1749757837000)]

In [0]:
# Get base parameters with ADF
dbutils.widgets.text("start_date", "")
dbutils.widgets.text("end_date", "")
start_date = dbutils.widgets.get("start_date")
end_date = dbutils.widgets.get("end_date")

' Data Factory\n# Get base parameters\ndbutils.widgets.text("start_date", "")\ndbutils.widgets.text("end_date", "")\nstart_date = dbutils.widgets.get("start_date")\nend_date = dbutils.widgets.get("end_date")\n'

In [0]:
# Construct the API URL with start and end dates provided by Data Factory, formatted for geojson output.
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try:
    # Make the GET request to fetch data
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    data = response.json().get('features', [])

    if not data:
        print("No data returned for the specified date range.")
    else:
        # Specify the ADLS path
        file_path = f"{bronze_adls}/{start_date}_earthquake_data.json"

        # Save the JSON data
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")


Wrote 321917 bytes.
Data successfully saved to abfss://bronze@dataearthquakee.dfs.core.windows.net//2025-06-12_earthquake_data.json


In [0]:

## Data Factory
# Define your variables
output_data = {
    "start_date": start_date,
    "end_date": end_date,
    "bronze_adls": bronze_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls
}

# Serialize the dictionary to a JSON string
output_json = json.dumps(output_data)

# Log the serialized JSON for debugging
print(f"Serialized JSON: {output_json}")

# Return the JSON string
dbutils.notebook.exit(output_json)
     

' Data Factory\n# Define your variables\noutput_data = {\n    "start_date": start_date,\n    "end_date": end_date,\n    "bronze_adls": bronze_adls,\n    "silver_adls": silver_adls,\n    "gold_adls": gold_adls\n}\n\n# Serialize the dictionary to a JSON string\noutput_json = json.dumps(output_data)\n\n# Log the serialized JSON for debugging\nprint(f"Serialized JSON: {output_json}")\n\n# Return the JSON string\ndbutils.notebook.exit(output_json)\n'